In [44]:
import random
import pickle
import pennylane as qml
import torch
from data_utils import mnist_preparation
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from tqdm import tqdm
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from pennylane import numpy as np

In [45]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                #transforms.Lambda(lambda img: add_salt_and_pepper_noise(img, salt_prob=0.1, pepper_prob=0.1)),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=64, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

In [46]:
with open("/Users/jackvittori/Desktop/no-noise/norumore-training.pickle", "rb") as file: 
    training_history = pickle.load(file)
loss_history = training_history['loss_history']
mcm_accuracy = training_history['mcm_accuracy']
fm_accuracy = training_history['fm_accuracy']
weights = training_history['model_params']

In [47]:
for param in weights.values():
    param.requires_grad = False

In [48]:
weights['layer_0']

In [49]:
"""with open("/Users/jackvittori/Desktop/4layerokkkk/weight4layer.pickle", "rb") as file: 
    weights_small = pickle.load(file)
for param in weights_small.values():
    param.requires_grad = False"""

In [50]:
weights

In [26]:
device = torch.device("mps")

# No noise 

In [57]:
p = 0.12
def early_evaluation_utils(state: torch.Tensor = None): 
    first_pair = [0,1]
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(wires = first_pair)

def fully_evaluation_utils(state: torch.Tensor = None):
    first_pair = [0,1]
    second_pair = [2,3]
    measurements = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
            
    for w in first_pair: 
        measurements.append(qml.measure(wires=w, reset=False, postselect=None))
    #m_0 = qml.measure(wires = 0, reset=False, postselect=None)
    #m_1 = qml.measure(wires = 1, reset=False, postselect=None)
    #print('ok meas 1')
    for i in range(4, 8):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(op = measurements), qml.probs(wires=[2,3])

mixed_device = qml.device("default.mixed", wires=[0,1,2,3,4,5,6,7,8,9])
late_qnode = qml.QNode(fully_evaluation_utils, mixed_device)
early_qnode = qml.QNode(early_evaluation_utils, mixed_device)

In [58]:
def evaluation_routine(dataloader: DataLoader, threshold: float):
    
    evaluation_results = []
    early_results = []
    count_1 = 0 #counter for early classified images
    count_2 = 0 #counter for final classified images
    early_correct = 0 #counter for correctly early classified images 
    final_correct = 0 #counter for correctly final classified images
    executed_layers = 0
    
    for i, (img, target) in tqdm(enumerate(dataloader.dataset)):
            
        img = img.type(torch.float64)
        img = img / torch.linalg.norm(img).view(-1, 1)
        #mid circuit evaluation
        early_probs = early_qnode(img)
        early_prediction = torch.argmax(early_probs)
        confidence = early_probs[early_prediction].item()
        early_guess = early_prediction == target
        early_results.append(early_guess.item())
        
        if confidence >= threshold:
            evaluation_results.append(early_guess.item())
            count_1 += 1
            executed_layers += 4
            if early_guess: 
                early_correct += 1
                
        else: 
            final_probs = late_qnode(img)
            early_full, final_full = final_probs
            final_predictions = torch.argmax(final_full)
            final_guess = final_predictions == target
            evaluation_results.append(final_guess.item())
            count_2 += 1
            executed_layers += 12
            if final_guess: 
                final_correct += 1
        
    total_accuracy = sum([1 for i in evaluation_results if i == True])/len(evaluation_results)
    early_total_accuracy = sum([1 for i in early_results if i == True])/len(early_results)
    early_exited_accuracy = early_correct/count_1 if count_1 > 0 else 0
    final_exited_accuracy = final_correct/count_2 if count_2 > 0 else 0   
    
    return total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers


In [59]:
import pandas as pd
def explain_evaluation(dataloader: DataLoader, threshold: List[float]):
    summary_data = {
    'Threshold': [],
    'Total Accuracy': [],
    '# early exited images': [],
    'Early exited Accuracy': [],
    'Early total accuracy': [],
    '# final classified images': [],
    'Final classified Accuracy': [],
    "Executed layers": []}
    
    for t in tqdm(threshold):
        total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers = evaluation_routine(dataloader, t)
        summary_data['Threshold'].append(t)
        summary_data['Total Accuracy'].append(total_accuracy)
        summary_data['# early exited images'].append(count_1)
        summary_data['Early exited Accuracy'].append(early_exited_accuracy)
        summary_data['Early total accuracy'].append(early_total_accuracy)
        summary_data['# final classified images'].append(count_2)
        summary_data['Final classified Accuracy'].append(final_exited_accuracy)
        summary_data['Executed layers'].append(executed_layers)
        
    df = pd.DataFrame(summary_data)
    return summary_data, df

In [60]:
thresholds = [0.2505, 0.2510, 0.2515, 0.2520, 0.2525, 0.2530, 0.2535]

In [61]:
[0.2505, 0.2507, 0.2510, 0.2513, 0.2515, 0.2520]

In [62]:
summary, table = explain_evaluation(test_dataloader, thresholds)

In [63]:
table

In [15]:
summary

In [64]:
ev_data = {
    'summary': summary,
    'table': table}
import pickle
with open("/Users/jackvittori/Desktop/DEPOLARIZING/tutte/noise.pickle", "wb") as file:
    pickle.dump(ev_data, file)

# Noise 0.005

In [15]:
p = 0.015

def early_evaluation_utils(state: torch.Tensor = None): 
    first_pair = [0,1]
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(wires = first_pair)

def fully_evaluation_utils(state: torch.Tensor = None):
    first_pair = [0,1]
    second_pair = [2,3]
    measurements = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
            
    for w in first_pair: 
        measurements.append(qml.measure(wires=w, reset=False, postselect=None))
    for i in range(4, 8):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(op = measurements), qml.probs(wires=[2,3])

mixed_device = qml.device("default.mixed", wires=[0,1,2,3,4,5,6,7,8,9])
late_qnode = qml.QNode(fully_evaluation_utils, mixed_device)
early_qnode = qml.QNode(early_evaluation_utils, mixed_device)

In [16]:
def evaluation_routine(dataloader: DataLoader, threshold: float):
    
    evaluation_results = []
    early_results = []
    count_1 = 0 #counter for early classified images
    count_2 = 0 #counter for final classified images
    early_correct = 0 #counter for correctly early classified images 
    final_correct = 0 #counter for correctly final classified images
    executed_layers = 0
    
    for i, (img, target) in tqdm(enumerate(dataloader.dataset)):
            
        img = img.type(torch.float64)
        img = img / torch.linalg.norm(img).view(-1, 1)
        
        #mid circuit evaluation
        early_probs = early_qnode(img)
        early_prediction = torch.argmax(early_probs)
        confidence = early_probs[early_prediction].item()
        early_guess = early_prediction == target
        early_results.append(early_guess.item())
        
        if confidence >= threshold:
            #print('early')
            evaluation_results.append(early_guess.item())
            count_1 += 1
            executed_layers += 4
            if early_guess: 
                early_correct += 1
                
        else: 
            #print('post')
            final_probs = late_qnode(img)
            early_full, final_full = final_probs
            final_predictions = torch.argmax(final_full)
            final_guess = final_predictions == target
            evaluation_results.append(final_guess.item())
            count_2 += 1
            executed_layers += 12
            if final_guess: 
                final_correct += 1
        
    total_accuracy = sum([1 for i in evaluation_results if i == True])/len(evaluation_results)
    early_total_accuracy = sum([1 for i in early_results if i == True])/len(early_results)
    early_exited_accuracy = early_correct/count_1 if count_1 > 0 else 0
    final_exited_accuracy = final_correct/count_2 if count_2 > 0 else 0   
    
    return total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers


In [17]:
import pandas as pd
def explain_evaluation(dataloader: DataLoader, threshold: List[float]):
    summary_data = {
    'Threshold': [],
    'Total Accuracy': [],
    '# early exited images': [],
    'Early exited Accuracy': [],
    'Early total accuracy': [],
    '# final classified images': [],
    'Final classified Accuracy': [],
    "Executed layers": []}
    
    for t in tqdm(threshold):
        total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers = evaluation_routine(dataloader, t)
        summary_data['Threshold'].append(t)
        summary_data['Total Accuracy'].append(total_accuracy)
        summary_data['# early exited images'].append(count_1)
        summary_data['Early exited Accuracy'].append(early_exited_accuracy)
        summary_data['Early total accuracy'].append(early_total_accuracy)
        summary_data['# final classified images'].append(count_2)
        summary_data['Final classified Accuracy'].append(final_exited_accuracy)
        summary_data['Executed layers'].append(executed_layers)
        
    df = pd.DataFrame(summary_data)
    return summary_data, df

In [18]:
thresholds = [0.26, 0.28, 0.30, 0.32, 0.34, 0.36]

In [19]:
summary, table = explain_evaluation(test_dataloader, thresholds)

In [20]:
ev_data = {
    'summary': summary,
    'table': table}
import pickle
with open("/Users/jackvittori/Desktop/DEPOLARIZING/tutte/mcm0-015.pickle", "wb") as file:
    pickle.dump(ev_data, file)

# Noise 0.01

In [21]:
p = 0.005

def early_evaluation_utils(state: torch.Tensor = None): 
    first_pair = [0,1]
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(wires = first_pair)

def fully_evaluation_utils(state: torch.Tensor = None):
    first_pair = [0,1]
    second_pair = [2,3]
    measurements = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
            
    for w in first_pair: 
        measurements.append(qml.measure(wires=w, reset=False, postselect=None))
    #m_0 = qml.measure(wires = 0, reset=False, postselect=None)
    #m_1 = qml.measure(wires = 1, reset=False, postselect=None)
    #print('ok meas 1')
    for i in range(4, 8):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(op = measurements), qml.probs(wires=[2,3])

mixed_device = qml.device("default.mixed", wires=[0,1,2,3,4,5,6,7,8,9])
late_qnode = qml.QNode(fully_evaluation_utils, mixed_device)
early_qnode = qml.QNode(early_evaluation_utils, mixed_device)

In [22]:
def evaluation_routine(dataloader: DataLoader, threshold: float):
    
    evaluation_results = []
    early_results = []
    count_1 = 0 #counter for early classified images
    count_2 = 0 #counter for final classified images
    early_correct = 0 #counter for correctly early classified images 
    final_correct = 0 #counter for correctly final classified images
    executed_layers = 0
    
    for i, (img, target) in tqdm(enumerate(dataloader.dataset)):
            
        img = img.type(torch.float64)
        img = img / torch.linalg.norm(img).view(-1, 1)
        
        #mid circuit evaluation
        early_probs = early_qnode(img)
        early_prediction = torch.argmax(early_probs)
        confidence = early_probs[early_prediction].item()
        early_guess = early_prediction == target
        early_results.append(early_guess.item())
        
        if confidence >= threshold:
            #print('early')
            evaluation_results.append(early_guess.item())
            count_1 += 1
            executed_layers += 4
            if early_guess: 
                early_correct += 1
                
        else: 
            #print('post')
            final_probs = late_qnode(img)
            early_full, final_full = final_probs
            final_predictions = torch.argmax(final_full)
            final_guess = final_predictions == target
            evaluation_results.append(final_guess.item())
            count_2 += 1
            executed_layers += 12
            if final_guess: 
                final_correct += 1
        
    total_accuracy = sum([1 for i in evaluation_results if i == True])/len(evaluation_results)
    early_total_accuracy = sum([1 for i in early_results if i == True])/len(early_results)
    early_exited_accuracy = early_correct/count_1 if count_1 > 0 else 0
    final_exited_accuracy = final_correct/count_2 if count_2 > 0 else 0   
    
    return total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers


In [23]:
import pandas as pd
def explain_evaluation(dataloader: DataLoader, threshold: List[float]):
    summary_data = {
    'Threshold': [],
    'Total Accuracy': [],
    '# early exited images': [],
    'Early exited Accuracy': [],
    'Early total accuracy': [],
    '# final classified images': [],
    'Final classified Accuracy': [],
    "Executed layers": []}
    
    for t in tqdm(threshold):
        total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers = evaluation_routine(dataloader, t)
        summary_data['Threshold'].append(t)
        summary_data['Total Accuracy'].append(total_accuracy)
        summary_data['# early exited images'].append(count_1)
        summary_data['Early exited Accuracy'].append(early_exited_accuracy)
        summary_data['Early total accuracy'].append(early_total_accuracy)
        summary_data['# final classified images'].append(count_2)
        summary_data['Final classified Accuracy'].append(final_exited_accuracy)
        summary_data['Executed layers'].append(executed_layers)
        
    df = pd.DataFrame(summary_data)
    return summary_data, df

In [24]:
thresholds = [0.26, 0.28, 0.30, 0.32, 0.34, 0.36]

In [25]:
summary, table = explain_evaluation(test_dataloader, thresholds)

In [26]:
table

In [27]:
ev_data = {
    'summary': summary,
    'table': table}
import pickle
with open("/Users/jackvittori/Desktop/DEPOLARIZING/tutte/mcm0-005.pickle", "wb") as file:
    pickle.dump(ev_data, file)

# Noise 0.015

In [26]:
p = 0.015

def early_evaluation_utils(state: torch.Tensor = None): 
    first_pair = [0,1]
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(wires = first_pair)

def fully_evaluation_utils(state: torch.Tensor = None):
    first_pair = [0,1]
    second_pair = [2,3]
    measurements = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
            
    for w in first_pair: 
        measurements.append(qml.measure(wires=w, reset=False, postselect=None))
    #m_0 = qml.measure(wires = 0, reset=False, postselect=None)
    #m_1 = qml.measure(wires = 1, reset=False, postselect=None)
    #print('ok meas 1')
    for i in range(4, 8):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(op = measurements), qml.probs(wires=[2,3])

mixed_device = qml.device("default.mixed", wires=[0,1,2,3,4,5,6,7,8,9])
late_qnode = qml.QNode(fully_evaluation_utils, mixed_device)
early_qnode = qml.QNode(early_evaluation_utils, mixed_device)

In [27]:
def evaluation_routine(dataloader: DataLoader, threshold: float):
    
    evaluation_results = []
    early_results = []
    count_1 = 0 #counter for early classified images
    count_2 = 0 #counter for final classified images
    early_correct = 0 #counter for correctly early classified images 
    final_correct = 0 #counter for correctly final classified images
    executed_layers = 0
    
    for i, (img, target) in tqdm(enumerate(dataloader.dataset)):
        if i == 500:
            break 
            
        img = img.type(torch.float64)
        img = img / torch.linalg.norm(img).view(-1, 1)
        
        #mid circuit evaluation
        early_probs = early_qnode(img)
        early_prediction = torch.argmax(early_probs)
        confidence = early_probs[early_prediction].item()
        early_guess = early_prediction == target
        early_results.append(early_guess.item())
        
        if confidence >= threshold:
            #print('early')
            evaluation_results.append(early_guess.item())
            count_1 += 1
            executed_layers += 4
            if early_guess: 
                early_correct += 1
                
        else: 
            #print('post')
            final_probs = late_qnode(img)
            early_full, final_full = final_probs
            final_predictions = torch.argmax(final_full)
            final_guess = final_predictions == target
            evaluation_results.append(final_guess.item())
            count_2 += 1
            executed_layers += 12
            if final_guess: 
                final_correct += 1
        
    total_accuracy = sum([1 for i in evaluation_results if i == True])/len(evaluation_results)
    early_total_accuracy = sum([1 for i in early_results if i == True])/len(early_results)
    early_exited_accuracy = early_correct/count_1 if count_1 > 0 else 0
    final_exited_accuracy = final_correct/count_2 if count_2 > 0 else 0   
    
    return total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers


In [28]:
import pandas as pd
def explain_evaluation(dataloader: DataLoader, threshold: List[float]):
    summary_data = {
    'Threshold': [],
    'Total Accuracy': [],
    '# early exited images': [],
    'Early exited Accuracy': [],
    'Early total accuracy': [],
    '# final classified images': [],
    'Final classified Accuracy': [],
    "Executed layers": []}
    
    for t in tqdm(threshold):
        total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers = evaluation_routine(dataloader, t)
        summary_data['Threshold'].append(t)
        summary_data['Total Accuracy'].append(total_accuracy)
        summary_data['# early exited images'].append(count_1)
        summary_data['Early exited Accuracy'].append(early_exited_accuracy)
        summary_data['Early total accuracy'].append(early_total_accuracy)
        summary_data['# final classified images'].append(count_2)
        summary_data['Final classified Accuracy'].append(final_exited_accuracy)
        summary_data['Executed layers'].append(executed_layers)
        
    df = pd.DataFrame(summary_data)
    return summary_data, df

In [29]:
thresholds = [0.26, 0.28, 0.30, 0.32, 0.34, 0.36]

In [30]:
summary, table = explain_evaluation(test_dataloader, thresholds)

In [31]:
table

In [32]:
ev_data = {
    'summary': summary,
    'table': table}
import pickle
with open("/Users/jackvittori/Desktop/DEPOLARIZING/500immagini/mcm015.pickle", "wb") as file:
    pickle.dump(ev_data, file)

# Noise 0.2

In [35]:
p = 0

def early_evaluation_utils(state: torch.Tensor = None): 
    first_pair = [0,1]
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(wires = first_pair)

def fully_evaluation_utils(state: torch.Tensor = None):
    first_pair = [0,1]
    second_pair = [2,3]
    measurements = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
            
    for w in first_pair: 
        measurements.append(qml.measure(wires=w, reset=False, postselect=None))
    #m_0 = qml.measure(wires = 0, reset=False, postselect=None)
    #m_1 = qml.measure(wires = 1, reset=False, postselect=None)
    #print('ok meas 1')
    for i in range(4, 8):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(op = measurements), qml.probs(wires=[2,3])

mixed_device = qml.device("default.mixed", wires=[0,1,2,3,4,5,6,7,8,9])
late_qnode = qml.QNode(fully_evaluation_utils, mixed_device)
early_qnode = qml.QNode(early_evaluation_utils, mixed_device)

In [36]:
def evaluation_routine(dataloader: DataLoader, threshold: float):
    
    evaluation_results = []
    early_results = []
    count_1 = 0 #counter for early classified images
    count_2 = 0 #counter for final classified images
    early_correct = 0 #counter for correctly early classified images 
    final_correct = 0 #counter for correctly final classified images
    executed_layers = 0
    
    for i, (img, target) in tqdm(enumerate(dataloader.dataset)):
        if i == 100:
            break 
            
        img = img.type(torch.float64)
        img = img / torch.linalg.norm(img).view(-1, 1)
        
        #mid circuit evaluation
        early_probs = early_qnode(img)
        early_prediction = torch.argmax(early_probs)
        confidence = early_probs[early_prediction].item()
        early_guess = early_prediction == target
        early_results.append(early_guess.item())
        
        if confidence >= threshold:
            #print('early')
            evaluation_results.append(early_guess.item())
            count_1 += 1
            executed_layers += 4
            if early_guess: 
                early_correct += 1
                
        else: 
            #print('post')
            final_probs = late_qnode(img)
            early_full, final_full = final_probs
            final_predictions = torch.argmax(final_full)
            final_guess = final_predictions == target
            evaluation_results.append(final_guess.item())
            count_2 += 1
            executed_layers += 12
            if final_guess: 
                final_correct += 1
        
    total_accuracy = sum([1 for i in evaluation_results if i == True])/len(evaluation_results)
    early_total_accuracy = sum([1 for i in early_results if i == True])/len(early_results)
    early_exited_accuracy = early_correct/count_1 if count_1 > 0 else 0
    final_exited_accuracy = final_correct/count_2 if count_2 > 0 else 0   
    
    return total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers


In [37]:
import pandas as pd
def explain_evaluation(dataloader: DataLoader, threshold: List[float]):
    summary_data = {
    'Threshold': [],
    'Total Accuracy': [],
    '# early exited images': [],
    'Early exited Accuracy': [],
    'Early total accuracy': [],
    '# final classified images': [],
    'Final classified Accuracy': [],
    "Executed layers": []}
    
    for t in tqdm(threshold):
        total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers = evaluation_routine(dataloader, t)
        summary_data['Threshold'].append(t)
        summary_data['Total Accuracy'].append(total_accuracy)
        summary_data['# early exited images'].append(count_1)
        summary_data['Early exited Accuracy'].append(early_exited_accuracy)
        summary_data['Early total accuracy'].append(early_total_accuracy)
        summary_data['# final classified images'].append(count_2)
        summary_data['Final classified Accuracy'].append(final_exited_accuracy)
        summary_data['Executed layers'].append(executed_layers)
        
    df = pd.DataFrame(summary_data)
    return summary_data, df

In [38]:
thresholds = [round(x * 0.01 + 0.26, 2) for x in range(25)]

In [39]:
summary, table = explain_evaluation(test_dataloader, thresholds)

In [40]:
table

In [41]:
ev_data = {
    'summary': summary,
    'table': table}
import pickle
with open("/Users/jackvittori/Desktop/DEPOLARIZING/analitica/depomcm02.pickle", "wb") as file:
    pickle.dump(ev_data, file)